## Get Training Features

### Training data were generated in file "Preprocess_i2b2_clinical_rel_dataset.ipynb" and pickled.

In [8]:
import pickle as pkl

filenm_train = 'train_features_02Aug2021'
train_features_LOL = pkl.load(open(filenm_train, 'rb'))

filenm_test = 'test_features_02Aug2021'
test_features_LOL = pkl.load(open(filenm_test, 'rb'))

In [13]:
len(train_features_LOL), len(test_features_LOL), train_features_LOL[0], test_features_LOL[0]

(650,
 350,
 [{'sent': 0,
   'word': 'vital',
   'prev_word': '*',
   'next_word': 'SIGNS',
   'word_length': 5,
   'prev_2_words': '*',
   'next_2_words': 'SIGNS -',
   'word_POS_tag': 'ADJ',
   'prev_POS_tag': '',
   'next_POS_tag': 'NOUN',
   'lemmatized': 'vital',
   'parent': '',
   'bigram': 'vital SIGNS',
   'trigram': '* vital SIGNS',
   'reverse_bigram': '* vital',
   'BOS': True,
   'EOS': False,
   'sty_word': 'Food',
   'sty_prev_word': '',
   'sty_next_word': 'Manufactured Object',
   'match_bigram': True,
   'match_trigram': False,
   'match_reverse_bigram': False,
   'CIU_word': 'C1875856'},
  {'sent': 0,
   'word': 'signs',
   'prev_word': 'VITAL',
   'next_word': '-',
   'word_length': 5,
   'prev_2_words': 'VITAL',
   'next_2_words': '- Temp',
   'word_POS_tag': 'NOUN',
   'prev_POS_tag': 'ADJ',
   'next_POS_tag': 'PUNCT',
   'lemmatized': 'sign',
   'parent': '',
   'bigram': 'signs -',
   'trigram': 'VITAL signs -',
   'reverse_bigram': 'VITAL signs',
   'BOS': Fals

## Get Lables

In [1]:
import pandas as pd

In [2]:
train_tagged = pd.read_csv("rpt_words_clean_train_02Aug2021.csv")
test_tagged = pd.read_csv("rpt_words_clean_test_02Aug2021.csv")

In [5]:
train_tagged.shape, test_tagged.shape

((16378, 5), (8503, 5))

In [41]:
#tagged = tagged.loc[0:4011]

In [42]:
#tagged['Tag'].fillna('O', inplace=True)

In [14]:
train_tagged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16378 entries, 0 to 16377
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  16378 non-null  int64 
 1   Sentence #  16378 non-null  int64 
 2   Word        16378 non-null  object
 3   POS         16378 non-null  object
 4   Tag         16378 non-null  object
dtypes: int64(2), object(3)
memory usage: 639.9+ KB


In [15]:
train_tagged.tail()

,Unnamed: 0,Sentence #,Word,POS,Tag
16373,16373,999,or,CCONJ,O
16374,16374,999,powders,NOUN,O
16375,16375,999,to,ADP,O
16376,16376,999,incisions,NOUN,B-problem
16377,16377,999,.,PUNCT,O


In [17]:
list(train_tagged.loc[train_tagged['Sentence #'] == 1, 'Tag'])

['O', 'B-treatment', 'O', 'O', 'O', 'O', 'O', 'B-problem', 'O']

In [18]:
def get_List(df, i):
    return list(df.loc[df['Sentence #'] == i, 'Tag'])

In [34]:
train_sentences_count = train_tagged['Sentence #'].unique()
test_sentences_count = test_tagged['Sentence #'].unique()

In [35]:
train_tag_list = []
for i in train_sentences_count:
    temp_lst = get_List(train_tagged, i)
    train_tag_list.append(temp_lst)
    
test_tag_list = []
for i in test_sentences_count:
    temp_lst = get_List(test_tagged, i)
    test_tag_list.append(temp_lst)

In [36]:
len(train_tag_list), len(test_tag_list)

(650, 350)

In [39]:
%%time
len(train_features_LOL), len(test_features_LOL)

Wall time: 0 ns


(650, 350)

## TRAIN TEST SPLIT

In [40]:
#Modeling
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
import scipy.stats
import eli5

Using TensorFlow backend.


In [52]:
#X_train, X_test, y_train, y_test = train_test_split(train_features, tag_list, test_size=0.33, random_state=0)

In [53]:
%%time
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=200,
          all_possible_transitions=False)
#We predcit using the same 5 fold cross validation
pred = cross_val_predict(estimator=crf, X=train_features_LOL, y=train_tag_list, cv=5)
#Lets evaluate the mode
report = flat_classification_report(y_pred=pred, y_true=tag_list)
print(report)

C:\Users\aasha\anaconda3\envs\tfgpu\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


              precision    recall  f1-score   support

           B       0.65      0.38      0.48       203
           I       0.70      0.42      0.52       189
           O       0.94      0.99      0.96      3620

    accuracy                           0.93      4012
   macro avg       0.76      0.59      0.66      4012
weighted avg       0.92      0.93      0.92      4012



<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\aasha\anaconda3\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [41]:
%%time
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True
)
crf.fit(train_features_LOL, train_tag_list)

Wall time: 11.1 s


C:\Users\aasha\anaconda3\envs\tfgpu\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=200)

In [42]:
filenm = 'crf_i2b2_trained.pickle'
#Step 1: Create or open a file with write-binary mode and save the model to it
pickle = pkl.dump(crf, open(filenm, 'wb'))

In [43]:
y_pred = crf.predict(test_features_LOL)
print(metrics.flat_classification_report(test_tag_list, y_pred))

              precision    recall  f1-score   support

   B-problem       0.44      0.13      0.20       275
      B-test       0.64      0.35      0.45        71
 B-treatment       0.50      0.07      0.12        74
   I-problem       0.40      0.19      0.25       357
      I-test       0.71      0.41      0.52       114
 I-treatment       0.29      0.03      0.05        79
           O       0.90      0.98      0.94      7533

    accuracy                           0.89      8503
   macro avg       0.55      0.31      0.36      8503
weighted avg       0.85      0.89      0.86      8503



C:\Users\aasha\anaconda3\envs\tfgpu\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [52]:
y_test = test_tag_list
X_test = test_features_LOL

In [53]:
for x,y in zip(X_test, y_test):
    for xi, yi in zip(x, y):
        print(xi.get('word'), '--', yi)
        

because -- O
of -- O
expected -- O
long -- O
term -- O
antibiotic -- O
administration -- O
, -- O
a -- O
picc -- O
line -- O
was -- O
placed -- O
and -- O
the -- O
patient -- O
was -- O
prepared -- O
for -- O
transfer -- O
back -- O
to -- O
fo -- O
hospital -- O
. -- O
she -- O
states -- O
this -- O
light-headedness -- B-problem
is -- O
often -- O
associated -- O
with -- O
shortness -- O
of -- O
breath -- O
and -- O
diaphoresis -- B-problem
occasionally -- O
with -- O
nausea -- O
. -- O
the -- O
patient -- O
&apos;s -- O
pain -- O
was -- O
initially -- O
controlled -- O
with -- O
iv -- O
pain -- O
medications -- O
but -- O
at -- O
the -- O
time -- O
of -- O
discharge -- O
to -- O
the -- O
rehab -- O
facility -- O
he -- O
was -- O
able -- O
to -- O
control -- O
his -- B-problem
pain -- I-problem
on -- O
po -- B-treatment
pain -- I-treatment
medications -- I-treatment
. -- O
it -- O
was -- O
felt -- O
that -- O
the -- O
patient -- O
&apos;s -- O
extensive -- O
pulmonary -- O
metastases -

of -- O
6 -- O
, -- O
wedge -- O
pressure -- O
of -- O
10 -- O
to -- O
12 -- O
, -- O
cardiac -- O
index -- O
of -- O
4.7 -- O
, -- O
and -- O
svr -- B-test
of -- O
438 -- O
all -- O
consistent -- O
with -- O
a -- B-problem
pseudomonas -- O
sepsis -- O
picture -- O
. -- O
2015-10-28 -- O
03:30 -- O
pm -- O
blood -- O
glucose -- O
- -- O
98 -- O
urean -- O
- -- O
26 -- O
* -- O
creat -- O
- -- O
1.6 -- O
* -- O
na -- O
- -- O
140 -- O
k -- O
- -- O
4.6 -- O
cl -- O
- -- O
104 -- O
hco3 -- O
- -- O
27 -- O
angap -- O
- -- O
14 -- O
elavil -- O
by -- O
mouth -- O
15 -- O
mg -- O
every -- O
night -- O
, -- O
dulcolax -- O
rectally -- O
10 -- O
mg -- O
once -- O
a -- O
day -- O
, -- O
tegretol -- O
by -- O
mouth -- O
100 -- O
mg -- O
twice -- O
a -- O
day -- O
, -- O
benadryl -- O
by -- O
mouth -- O
25 -- O
mg -- O
three -- O
times -- O
a -- O
day -- O
, -- O
colace -- O
by -- O
mouth -- O
200 -- O
mg -- O
twice -- O
a -- O
day -- O
, -- O
vasotec -- O
by -- O
mouth -- O
10 -- O
mg -- O
twi

pco2 -- O
- -- O
39 -- O
ph -- O
- -- O
7.41 -- O
total -- O
co2 -- O
- -- O
26 -- O
base -- O
xs -- O
- -- O
0 -- O
aado2 -- O
- -- O
279 -- O
req -- O
o2 -- O
- -- O
52 -- O
- -- O
assist -- O
/ -- O
con -- O
intubated -- O
- -- O
intubated -- O
he -- O
underwent -- O
a -- B-test
staging -- I-test
ct -- I-test
scan -- I-test
torso -- I-test
which -- O
suggested -- O
no -- O
significant -- B-problem
increase -- I-problem
in -- I-problem
tumor -- I-problem
burden -- I-problem
. -- O
upon -- O
further -- O
review -- O
of -- O
ct -- O
and -- O
angiography -- O
results -- O
with -- O
neuroradiology -- O
and -- O
neurology -- O
service -- O
, -- O
it -- O
was -- O
decided -- O
that -- O
ct -- O
myelography -- O
was -- O
not -- O
indicated -- O
at -- O
this -- O
time -- O
. -- O
in -- O
the -- O
deanna -- O
, -- O
tmax -- O
102.8 -- O
, -- O
bp -- O
100 -- O
- -- O
140/60 -- O
s, -- O
hr -- O
80 -- O
s, -- O
rr -- O
20 -- O
s, -- O
sat -- O
75% -- O
on -- O
ra -- O
in -- O
triage -- O
, -- 

11.8 -- O
and -- O
an -- O
albumin -- O
of -- O
2.3 -- O
on -- O
admission -- O
. -- O
lopressor -- O
100 -- O
mg -- O
p.o. -- O
b.i.d. -- O
; -- O
nifedipine -- O
10 -- O
mg -- O
p.o. -- O
t.i.d. -- O
; -- O
timoptic -- O
solution -- O
0.5% -- O
os -- O
b.i.d. -- O
; -- O
coumadin -- O
1 -- O
mg -- O
alternating -- O
with -- O
1.5 -- O
mg -- O
q.o.d. -- O
at -- O
q.h.s. -- O
and -- O
haldol -- O
2 -- O
mg -- O
p.o. -- O
q.h.s. -- O
p.r.n. -- O
in -- O
the -- O
evening -- O
, -- O
only -- O
for -- O
agitation -- B-problem
associated -- O
with -- O
confusion -- B-problem
. -- O
physical -- O
examination -- O
revealed -- O
temperature -- O
was -- O
96.9 -- O
, -- O
heart -- O
rate -- O
was -- O
121 -- O
, -- O
blood -- O
pressure -- O
was -- O
122/86 -- O
, -- O
respiratory -- O
rate -- O
was -- O
22 -- O
, -- O
and -- O
oxygen -- O
saturation -- O
was -- O
96% -- O
on -- O
room -- O
air -- O
. -- O
hypertension -- O
was -- O
managed -- O
with -- O
beta -- O
blocker -- O
and -- O
ace -- 

decreased -- O
significantly -- O
. -- O
the -- O
patient -- O
was -- O
found -- O
to -- O
have -- O
5057 -- O
e. -- O
sixth -- O
blvd. -- O
elevations -- O
in -- O
i -- O
, -- O
avl -- O
, -- O
and -- O
v1 -- O
through -- O
v2 -- O
and -- O
st -- O
depressions -- O
in -- O
ii -- O
, -- O
iii -- O
, -- O
and -- O
avf -- O
. -- O
in -- O
this -- O
2 -- O
week -- O
interval -- O
he -- O
denied -- O
having -- O
cp -- O
/ -- O
chest -- O
discomfort -- O
or -- O
difficulty -- O
breathing -- O
, -- O
no -- O
cough -- O
or -- O
hemoptysis -- O
. -- O
second -- O
, -- O
he -- O
had -- O
a -- B-problem
witnessed -- I-problem
episode -- I-problem
in -- O
which -- O
his -- O
eyes -- O
rolled -- O
to -- O
the -- O
back -- O
of -- O
his -- O
head -- O
and -- O
there -- O
was -- O
a -- O
question -- O
of -- O
some -- B-problem
seizure -- I-problem
activity -- I-problem
. -- O
clonidine -- O
2.5 -- O
milligrams -- O
topical -- O
q. -- O
week -- O
, -- O
nitropaste -- O
2 -- O
inches -- O
q.4h. -- O
,

...

. -- O
24. -- O
warfarin -- O
1 -- O
mg -- O
tablet -- O
sig -- O
: -- O
one -- O
( -- O
1 -- O
) -- O
tablet -- O
po -- O
at -- O
bedtime -- O
: -- O
please -- O
monitor -- O
inr -- O
with -- O
goal -- O
02-24 -- O
. -- O
2015-10-28 -- O
03:30 -- O
pm -- O
blood -- O
glucose -- O
- -- O
98 -- O
urean -- O
- -- O
26 -- O
* -- O
creat -- O
- -- O
1.6 -- O
* -- O
na -- O
- -- O
140 -- O
k -- O
- -- O
4.6 -- O
cl -- O
- -- O
104 -- O
hco3 -- O
- -- O
27 -- O
angap -- O
- -- O
14 -- O
the -- O
cause -- O
of -- O
death -- B-problem
was -- O
a -- O
combination -- O
of -- O
gi -- O
bleeding -- O
felt -- O
to -- O
be -- O
ultimately -- O
related -- O
to -- O
his -- B-problem
known -- I-problem
metastatic -- I-problem
pancreatic -- I-problem
cancer -- I-problem
. -- O
the -- O
hypotension -- O
was -- O
likely -- O
due -- O
to -- O
volume -- B-problem
depletion -- I-problem
caused -- O
by -- O
her -- B-problem
diarrhea -- I-problem
/ -- O
loose -- O
stools -- O
in -- O
addition -- O
to -- O
decr

was -- O
sensitive -- O
to -- O
gentamicin -- O
and -- O
clindamycin -- B-treatment
. -- O
the -- O
patient -- O
was -- O
switched -- O
to -- O
ampicillin -- O
, -- O
gentamicin -- B-treatment
, -- O
and -- O
clindamycin -- O
, -- O
according -- O
to -- O
the -- O
culture -- O
, -- O
enterococci -- B-problem
and -- O
staphylococcus -- O
in -- O
the -- O
wound -- O
. -- O
the -- O
patient -- O
denies -- O
any -- O
chest -- O
pain -- O
, -- O
shortness -- O
of -- O
breath -- O
, -- O
dysuria -- O
, -- O
diarrhea -- O
, -- O
bright -- O
red -- O
blood -- O
per -- O
rectum -- O
. -- O
mr. -- O
eugene -- O
is -- O
a -- O
67 -- O
year -- O
old -- O
male -- O
with -- O
a -- O
remote -- O
history -- O
of -- O
chronic -- O
pancreatitis -- O
due -- O
to -- O
alcohol -- O
abuse -- O
30 -- O
years -- O
ago -- O
, -- O
who -- O
is -- O
transferred -- O
now -- O
from -- O
rehab -- O
hospital -- O
of -- O
the -- O
cape -- O
& -- O
islands -- O
hospital -- O
for -- O
an -- B-test
ercp -- I-test
and --

diagnosed -- O
recently -- O
with -- O
pancreatic -- O
cancer -- O
after -- O
he -- O
presented -- O
to -- O
seph&apos;sju -- O
memorial -- O
after -- O
a -- O
motor -- O
vehicle -- O
accident -- O
. -- O
includes -- O
recent -- O
infarct -- O
to -- O
pons -- O
, -- O
mid -- O
brain -- O
, -- O
left -- O
thalamus -- O
, -- O
bilateral -- O
temporal -- O
lobes -- O
and -- O
left -- O
visual -- O
cortex -- O
, -- O
on -- O
coumadin -- O
. -- O
f -- O
/ -- O
u -- O
with -- O
pcp -- O
and -- O
dr. -- O
pump -- O
as -- O
scheduled -- O
, -- O
return -- O
to -- O
ed -- O
with -- O
worsening -- O
sob -- O
or -- O
increased -- O
cough -- O
or -- O
sputum -- O
production -- O
radiographs -- B-test
demonstrate -- O
right -- B-problem
hip -- O
osteoarthritis -- O
. -- O
she -- O
also -- O
developed -- O
what -- O
appeared -- O
to -- O
be -- O
urinary -- B-problem
tract -- I-problem
infection -- I-problem
and -- O
was -- O
placed -- O
on -- O
po -- B-treatment
bactrim -- I-treatment
. -- O
6. -- O

fragmin -- O
for -- O
dvt -- O
prophylaxis -- O
and -- O
the -- O
patient -- O
was -- O
also -- O
started -- O
on -- O
dilantin -- O
levels -- O
. -- O
work-up -- O
for -- O
the -- O
source -- O
of -- O
the -- O
primary -- O
tumor -- O
which -- O
is -- O
still -- O
unknown -- O
included -- O
a -- B-test
mammogram -- I-test
which -- O
identified -- O
no -- O
suspicious -- O
microcalcifications -- O
and -- O
no -- O
dominant -- B-problem
masses -- I-problem
. -- O
# -- O
esrd -- B-problem
: -- O
etiology -- O
likely -- O
02-08 -- O
dm -- O
and -- O
htn -- B-problem
, -- O
pt -- O
tolerating -- O
hd -- O
well -- O
, -- O
and -- O
underwent -- O
dialysis -- O
without -- O
difficulty -- O
on -- O
01-24 -- O
. -- O
thickening -- B-problem
consistent -- O
with -- O
trisomy -- B-problem
21 -- O
in -- O
the -- O
b -- O
co-twin -- O
. -- O
fatigue -- B-problem
secondary -- O
to -- O
worsening -- B-problem
dyspnea -- I-problem
. -- O
there -- O
is -- O
a -- B-problem
persistent -- I-problem
flow 

persistent -- O
branch -- O
of -- O
the -- O
circumflex -- O
to -- O
pa -- O
branch -- O
fistula -- O
which -- O
anatomically -- O
were -- O
not -- O
suitable -- O
for -- O
coil -- B-treatment
embolization -- I-treatment
. -- O
neuro -- O
: -- O
no -- O
asterixis -- O
, -- O
cn -- O
ii-xii -- O
intact -- O
, -- O
5/5 -- O
strength -- O
throughout -- O
, -- O
sensation -- O
to -- O
lt -- O
intact -- O
throughout -- O
, -- O
gait -- O
deferred -- O
blood -- O
pressure -- O
156/67 -- O
, -- O
pulse -- O
80 -- O
and -- O
respiratory -- O
rate -- O
14 -- O
. -- O


In [55]:
def parse_output(X, y):
    sentences = []
    for x, y in zip(X, y):
        each_sent = " ".join((xi.get('word') + '/' + yi) for xi, yi in zip(x, y))
        sentences.append(each_sent)
    return sentences

parse_output(X_test, y_pred)

['because/O of/O expected/O long/O term/O antibiotic/O administration/O ,/O a/O picc/O line/O was/O placed/O and/O the/O patient/O was/O prepared/O for/O transfer/O back/O to/O fo/O hospital/O ./O',
 'she/O states/O this/O light-headedness/O is/O often/O associated/O with/O shortness/O of/O breath/O and/O diaphoresis/O occasionally/O with/O nausea/O ./O',
 'the/B-problem patient/I-problem &apos;s/I-problem pain/I-problem was/O initially/O controlled/O with/O iv/B-treatment pain/I-treatment medications/I-treatment but/O at/O the/O time/O of/O discharge/O to/O the/O rehab/O facility/O he/O was/O able/O to/O control/O his/O pain/O on/O po/O pain/O medications/O ./O',
 'it/O was/O felt/O that/O the/O patient/O &apos;s/O extensive/B-problem pulmonary/I-problem metastases/I-problem and/I-problem bilateral/I-problem pleural/I-problem effusions/I-problem resulted/O in/O increased/O work/O of/O respiration/O and/O a/O sensation/O of/O shortness/O of/O breath/O ./O',
 'her/O urine/O output/O dec

In [56]:
def parse_output_tuples(X, Y):
    sent = []
    for xi, yi in zip(X, Y):
        sent.append(list(map(lambda x, y: (x.get('word'), y), xi, yi)))
    return sent

parse_output_tuples(X_test, y_pred)

[[('because', 'O'),
  ('of', 'O'),
  ('expected', 'O'),
  ('long', 'O'),
  ('term', 'O'),
  ('antibiotic', 'O'),
  ('administration', 'O'),
  (',', 'O'),
  ('a', 'O'),
  ('picc', 'O'),
  ('line', 'O'),
  ('was', 'O'),
  ('placed', 'O'),
  ('and', 'O'),
  ('the', 'O'),
  ('patient', 'O'),
  ('was', 'O'),
  ('prepared', 'O'),
  ('for', 'O'),
  ('transfer', 'O'),
  ('back', 'O'),
  ('to', 'O'),
  ('fo', 'O'),
  ('hospital', 'O'),
  ('.', 'O')],
 [('she', 'O'),
  ('states', 'O'),
  ('this', 'O'),
  ('light-headedness', 'O'),
  ('is', 'O'),
  ('often', 'O'),
  ('associated', 'O'),
  ('with', 'O'),
  ('shortness', 'O'),
  ('of', 'O'),
  ('breath', 'O'),
  ('and', 'O'),
  ('diaphoresis', 'O'),
  ('occasionally', 'O'),
  ('with', 'O'),
  ('nausea', 'O'),
  ('.', 'O')],
 [('the', 'B-problem'),
  ('patient', 'I-problem'),
  ('&apos;s', 'I-problem'),
  ('pain', 'I-problem'),
  ('was', 'O'),
  ('initially', 'O'),
  ('controlled', 'O'),
  ('with', 'O'),
  ('iv', 'B-treatment'),
  ('pain', 'I-treatm

## Visualization of the result:

In [57]:
import eli5

In [58]:
eli5.show_weights(crf, top=3)

C:\Users\aasha\anaconda3\envs\tfgpu\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [61]:
eli5.show_weights(crf, top=10, targets=['O', 'B-problem', 'I-problem', 'B-test', 'I-test', 'B-treatment', 'I-treatment'])

C:\Users\aasha\anaconda3\envs\tfgpu\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
